# **Notebook 4.3: 6G Intelligent Beamforming with QML**

**Objective:** This notebook introduces Quantum Machine Learning (QML) by applying a **Quantum Support Vector Classifier (QSVC)** to a simulated **intelligent beamforming** problem. We will generate a synthetic dataset, find the optimal model complexity through hyperparameter tuning, and train both a classical SVM and a QSVC to compare their performance. 🧠

### **1. The Telecom Problem: Intelligent Beamforming**

In 6G networks, base stations use beamforming to precisely direct radio signals to users. The optimal beam configuration (e.g., its angle and width) depends on complex factors like user location, obstacles, and signal reflections. This can be framed as a **classification problem**: based on input data (like user coordinates and channel state information), the system must choose the best beam configuration from a predefined set.

We will simulate a scenario with two classes of users, each requiring a different beamforming setup. Our goal is to train a model that can accurately classify a new user and assign the correct beam.

---

### **2. Setup: Importing Libraries**

In [ ]:
# Core Python libraries
import numpy as np
import matplotlib.pyplot as plt

# Scikit-learn for classical ML and data generation
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Qiskit imports
from qiskit.primitives import Sampler
from qiskit.circuit.library import ZZFeatureMap
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Set a seed for reproducibility
np.random.seed(42)

---

### **3. Generating a Synthetic Beamforming Dataset**

We will create a synthetic dataset where the features could represent user coordinates or channel characteristics. The dataset is intentionally made non-linearly separable to demonstrate the power of kernel methods like SVM and QSVC.

In [ ]:
# Generate a dataset with 2 features for easy visualization
X, y = make_classification(
    n_samples=100,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    random_state=42,
    n_clusters_per_class=2
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Plot the dataset
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
plt.title("Synthetic Beamforming Dataset")
plt.xlabel("Feature 1 (e.g., User Azimuth)")
plt.ylabel("Feature 2 (e.g., Signal Strength)")
plt.show()

---

### **4. Classical Benchmark: Support Vector Machine (SVM)**

First, let's train a classical SVM with a standard Radial Basis Function (RBF) kernel to see how it performs on our dataset. This will be our baseline.

In [ ]:
# Create and train the classical SVM
classical_svm = SVC(kernel='rbf')
classical_svm.fit(X_train, y_train)

# Make predictions
classical_preds = classical_svm.predict(X_test)
classical_accuracy = accuracy_score(y_test, classical_preds)

print(f"Classical SVM Accuracy: {classical_accuracy * 100:.2f}%")

---

### **5. Quantum Approach: Hyperparameter Tuning for QSVC**

Instead of guessing the best complexity for our quantum model, we will systematically test a range of values for the `reps` hyperparameter in our `ZZFeatureMap`. This allows us to find the 'sweet spot' that avoids both underfitting and overfitting.

In [ ]:
accuracies = {}
reps_to_test = [1, 2, 3, 4, 5] # Test a range of complexity values

for reps_val in reps_to_test:
    print(f"--- Testing with reps = {reps_val} ---")
    
    # 1. Define the Feature Map
    num_features = X.shape[1]
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=reps_val)

    # 2. Define the Sampler and Fidelity object
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler=sampler)

    # 3. Instantiate the Quantum Kernel
    quantum_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

    # 4. Instantiate and Train the QSVC
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(X_train, y_train)

    # 5. Make predictions and store accuracy
    qsvc_preds = qsvc.predict(X_test)
    accuracies[reps_val] = accuracy_score(y_test, qsvc_preds)
    print(f"Accuracy for reps = {reps_val}: {accuracies[reps_val] * 100:.2f}%")

# --- Plot the tuning results ---
plt.figure(figsize=(10, 6))
plt.plot(list(accuracies.keys()), list(accuracies.values()), marker='o', linestyle='--')
plt.title("QSVC Accuracy vs. Feature Map Repetitions ('reps')")
plt.xlabel("Number of Reps")
plt.ylabel("Accuracy on Test Set")
plt.grid(True)
plt.show()

# --- Find the best result and re-train the model for visualization ---
best_reps = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_reps]
print(f"\nBest result found with reps = {best_reps}, achieving an accuracy of {best_accuracy * 100:.2f}%.")

print(f"\nRe-training the best model with reps = {best_reps} for visualization...")
best_feature_map = ZZFeatureMap(feature_dimension=num_features, reps=best_reps)
best_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=best_feature_map)
qsvc = QSVC(quantum_kernel=best_kernel)
qsvc.fit(X_train, y_train)
qsvc_preds = qsvc.predict(X_test) # This creates the variable for the next cell
print("Best model is ready for plotting.")

---

### **6. Visualizing the Best Model's Predictions**

Now we visualize the performance of our best-performing QSVC model. Instead of plotting the full decision boundary (which is computationally explosive), we will plot the test data points and highlight any points that the model misclassified. This is a fast and effective way to see the model's performance.

In [ ]:
plt.figure(figsize=(10, 8))

# Plot the test data points, colored by their true label
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.coolwarm, edgecolors='k', label='True Labels')

# Find where the QSVC made incorrect predictions
incorrect_indices = np.where(y_test != qsvc_preds)[0]

# Highlight the incorrect predictions with large 'X' markers
plt.scatter(
    X_test[incorrect_indices, 0],
    X_test[incorrect_indices, 1],
    s=250,
    facecolors='none',
    edgecolors='black',
    linewidths=2,
    marker='X',
    label='Incorrect Predictions'
)

# Use the 'best_accuracy' variable in the title
plt.title(f'QSVC Predictions on Test Data (Best Accuracy: {best_accuracy*100:.2f}%)')
plt.xlabel('Feature 1 (e.g., User Azimuth)')
plt.ylabel('Feature 2 (e.g., Signal Strength)')
plt.legend()
plt.show()

### **7. Conclusion and Telecom Impact**

In this notebook, we implemented a Quantum Support Vector Classifier and performed hyperparameter tuning to find the optimal model complexity. We compared this to a classical SVM on a simulated beamforming classification task.

The true potential for quantum advantage with QML lies in problems with a very high number of features, where classical computers struggle to find patterns. In 6G, this could include:

* **Complex Channel State Information (CSI):** Using dozens or hundreds of CSI parameters as features to predict the optimal beam configuration.
* **Dynamic Spectrum Access:** Classifying spectrum availability based on complex, high-dimensional interference data.
* **Network Security:** Detecting sophisticated attacks by classifying network traffic patterns with many features.

This notebook provides the foundational understanding of how to build and tune QML models for future intelligent and autonomous telecom networks.